In [1]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [62]:
consumerKey = "gbe6s4lSt3S9tehFu3vsGgy17"
consumerSecret = "1bGZQfOdp8to1B5yx46wjkpvfwaHOZ3o3CxhnNEJ7KEPm7YQ1X"
accessToken = "2740237440-0Jp61ts7goF1TxmrZL0P3iYo1yQ6RBYBeVFv4Op"
accessTokenSecret = "Ynpv9s62hkPYTJT7JaPq9Fsbp8Td14YigEC9yxODdWTDU"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [63]:
#Sentiment Analysis
def percentage(part,whole):
     return 100 * float(part)/float(whole)
keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = int(input ("Please enter how many tweets to analyze: "))
tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
 
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
 
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

Please enter keyword or hashtag to search: dogecoin
Please enter how many tweets to analyze: 2000


In [64]:
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  2000
positive number:  773
negative number:  267
neutral number:  960


In [65]:
tweet_list

,0
0,RT @KriptoUzmanim: 1 Kişiye 5000 #Raca Çekiliş...
1,RT @Oonabot: Every new Shibe's first day in Cr...
2,@TheCeloDoge Awesome project with great potent...
3,RT @CryptoInu_ABCD: #CryptoInu is about to fly...
4,RT @DogeWofficial: NEW TOP-DOG ON THE BLOCK! 💪...
...,...
1995,RT @togo_crypto: BUY #BabyDogeCoin NOW OR HAVE...
1996,RT @MattWallace888: Press RETWEET if you are e...
1997,RT @MattWallace888: Press RETWEET if you are e...
1998,@Ki99426219 @lemanaha90 @Rafikioktaviant \n\n#...


In [66]:
tweet_list.drop_duplicates(inplace = True)

In [67]:
tweet_list.to_csv('../03project/original.csv')

# clean data

In [68]:
tweets = pd.read_csv('../03project/original.csv').drop(columns=['Unnamed: 0'])

In [69]:
#Cleaning Text (RT, Punctuation etc)
#Creating new dataframe and new features
tweets["text"] = tweets['0']
tweets

,0,text
0,RT @KriptoUzmanim: 1 Kişiye 5000 #Raca Çekiliş...,RT @KriptoUzmanim: 1 Kişiye 5000 #Raca Çekiliş...
1,RT @Oonabot: Every new Shibe's first day in Cr...,RT @Oonabot: Every new Shibe's first day in Cr...
2,@TheCeloDoge Awesome project with great potent...,@TheCeloDoge Awesome project with great potent...
3,RT @CryptoInu_ABCD: #CryptoInu is about to fly...,RT @CryptoInu_ABCD: #CryptoInu is about to fly...
4,RT @DogeWofficial: NEW TOP-DOG ON THE BLOCK! 💪...,RT @DogeWofficial: NEW TOP-DOG ON THE BLOCK! 💪...
...,...,...
832,@elonmusk I wish I could get it's subscription...,@elonmusk I wish I could get it's subscription...
833,RT @ShawnDoge: Me once these charts start heat...,RT @ShawnDoge: Me once these charts start heat...
834,RT @ltngbolt1: #doge #dogecoin #dogearmy\n\nDo...,RT @ltngbolt1: #doge #dogecoin #dogearmy\n\nDo...
835,RT @TheCeloDoge: Hey @dogecoin #dogeUncle #Dog...,RT @TheCeloDoge: Hey @dogecoin #dogeUncle #Dog...


In [71]:
#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+) | (\w+:\/\/\S+)"," ",x)
# tweets["text"] = tweets.text.map(remove_rt)
tweets['text'] = tweets.text.map(rt)
tweets["text"] = tweets.text.str.lower()
tweets.head(10)

,0,text
0,RT @KriptoUzmanim: 1 Kişiye 5000 #Raca Çekiliş...,rt @kriptouzmanim: 1 kişiye 5000 #raca çekiliş...
1,RT @Oonabot: Every new Shibe's first day in Cr...,rt @oonabot: every new shibe's first day in cr...
2,@TheCeloDoge Awesome project with great potent...,awesome project with great potential. the ded...
3,RT @CryptoInu_ABCD: #CryptoInu is about to fly...,rt @cryptoinu_abcd: #cryptoinu is about to fly...
4,RT @DogeWofficial: NEW TOP-DOG ON THE BLOCK! 💪...,rt @dogewofficial: new top-dog on the block! 💪...
5,RT @DogzToken: 🔥 $DOGZ is now available on #Un...,rt @dogztoken: 🔥 $dogz is now available on #un...
6,RT @WhalexW: @whale_alert @TimerrOfficial is ...,"rt @whalexw: @whale_alert is the next #doge,..."
7,Dogecoin Energy ⚡️ https://t.co/PbL1OvrK94,dogecoin energy ⚡️
8,@TheCeloDoge Quite brilliant &amp; interesting...,quite brilliant &amp; interesting project at ...
9,#digitalmarketing #affiliate #travel #tourism ...,#digitalmarketing #affiliate #travel #tourism ...


In [73]:
#Calculating Negative, Positive, Neutral and Compound values
tweets[['polarity', 'subjectivity']] = tweets['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tweets['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tweets.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tweets.loc[index, 'sentiment'] = "positive"
    else:
        tweets.loc[index, 'sentiment'] = "neutral"
        tweets.loc[index, 'neg'] = neg
        tweets.loc[index, 'neu'] = neu
        tweets.loc[index, 'pos'] = pos
        tweets.loc[index, 'compound'] = comp
tweets.head(10)

,0,text,polarity,subjectivity,sentiment,neg,compound,neu,pos
0,RT @KriptoUzmanim: 1 Kişiye 5000 #Raca Çekiliş...,rt @kriptouzmanim: 1 kişiye 5000 #raca çekiliş...,0.000000,0.000000,positive,NaN,NaN,NaN,NaN
1,RT @Oonabot: Every new Shibe's first day in Cr...,rt @oonabot: every new shibe's first day in cr...,0.193182,0.393939,neutral,0.0,0.0,1.0,0.0
2,@TheCeloDoge Awesome project with great potent...,awesome project with great potential. the ded...,0.600000,0.912500,positive,NaN,NaN,NaN,NaN
3,RT @CryptoInu_ABCD: #CryptoInu is about to fly...,rt @cryptoinu_abcd: #cryptoinu is about to fly...,0.450000,0.600000,positive,NaN,NaN,NaN,NaN
4,RT @DogeWofficial: NEW TOP-DOG ON THE BLOCK! 💪...,rt @dogewofficial: new top-dog on the block! 💪...,0.170455,0.454545,negative,NaN,NaN,NaN,NaN
5,RT @DogzToken: 🔥 $DOGZ is now available on #Un...,rt @dogztoken: 🔥 $dogz is now available on #un...,0.400000,0.400000,neutral,0.0,0.0,1.0,0.0
6,RT @WhalexW: @whale_alert @TimerrOfficial is ...,"rt @whalexw: @whale_alert is the next #doge,...",0.000000,0.000000,neutral,0.0,0.0,1.0,0.0
7,Dogecoin Energy ⚡️ https://t.co/PbL1OvrK94,dogecoin energy ⚡️,0.000000,0.000000,positive,NaN,NaN,NaN,NaN
8,@TheCeloDoge Quite brilliant &amp; interesting...,quite brilliant &amp; interesting project at ...,0.425000,0.600000,positive,NaN,NaN,NaN,NaN
9,#digitalmarketing #affiliate #travel #tourism ...,#digitalmarketing #affiliate #travel #tourism ...,0.000000,0.000000,neutral,0.0,0.0,1.0,0.0


# crypto market

In [2]:
import pandas as pd

In [3]:
dogecoin_price = pd.read_csv('../sentiment_analysis/DOGE-USD.csv')
dogecoin_price

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-01,0.002346,0.002392,0.002322,0.002392,0.002392,17364744
1,2019-01-02,0.002388,0.002458,0.002372,0.002407,0.002407,18015392
2,2019-01-03,0.002404,0.002414,0.002356,0.002360,0.002360,17619234
3,2019-01-04,0.002364,0.002397,0.002274,0.002315,0.002315,19530100
4,2019-01-05,0.002320,0.002350,0.002280,0.002319,0.002319,17148586
...,...,...,...,...,...,...,...
1123,2022-01-28,0.141244,0.142413,0.138380,0.141656,0.141656,506667466
1124,2022-01-29,0.141649,0.143984,0.141480,0.143056,0.143056,397610776
1125,2022-01-30,0.143057,0.143515,0.138754,0.139471,0.139471,363976741
1126,2022-01-31,0.139469,0.142264,0.136853,0.141805,0.141805,412307174


In [4]:
gamestop_price = pd.read_csv('../sentiment_analysis/GME.csv')
gamestop_price

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,17.959999,18.290001,17.780001,18.260000,15.953856,2832700
1,2018-01-03,18.290001,18.370001,17.920000,18.200001,15.901434,3789200
2,2018-01-04,18.200001,18.379999,17.959999,18.320000,16.006279,2781300
3,2018-01-05,18.379999,18.730000,18.219999,18.680000,16.320812,3019000
4,2018-01-08,18.799999,19.400000,18.799999,19.230000,16.801350,3668400
...,...,...,...,...,...,...,...
1023,2022-01-25,96.919998,104.970001,95.010002,99.790001,99.790001,3574300
1024,2022-01-26,101.099998,119.000000,100.309998,103.260002,103.260002,8332500
1025,2022-01-27,104.139999,107.000000,92.889999,93.519997,93.519997,3661100
1026,2022-01-28,95.400002,98.000000,87.610001,97.910004,97.910004,4107600
